<a href="https://colab.research.google.com/github/mess-alina/Rec-Sys/blob/Course-Work/Course_work_Vorobeva_AN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Курсовой проект Воробьевой А. Н. по курсу "Рекомендательные системы"


In [1]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 17.6 MB/s eta 0:00:00


In [2]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808331 sha256=6b8ea0e9e18d91e26ca7530dc45db22255a842a4a96e70dbe4337f502672dc96
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
from implicit.bpr import BayesianPersonalizedRanking
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

# Модель второго уровня
from lightgbm import LGBMClassifier

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [4]:
# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS

    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """

    def __init__(self, data, weighting=True):

        # Топ покупок каждого юзера
        self.top_purchases = data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        self.top_purchases.sort_values('quantity', ascending=False, inplace=True)
        self.top_purchases = self.top_purchases[self.top_purchases['item_id'] != 999999]

        # Топ покупок по всему датасету
        self.overall_top_purchases = data.groupby('item_id')['quantity'].count().reset_index()
        self.overall_top_purchases.sort_values('quantity', ascending=False, inplace=True)
        self.overall_top_purchases = self.overall_top_purchases[self.overall_top_purchases['item_id'] != 999999]
        self.overall_top_purchases = self.overall_top_purchases.item_id.tolist()

        self.user_item_matrix = self._prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, \
            self.itemid_to_id, self.userid_to_id = self._prepare_dicts(self.user_item_matrix)

        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T

        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)

    @staticmethod
    def _prepare_matrix(data):
        """Готовит user-item матрицу"""
        user_item_matrix = pd.pivot_table(data,
                                          index='user_id', columns='item_id',
                                          values='quantity',  # Можно пробовать другие варианты
                                          aggfunc='count',
                                          fill_value=0
                                          )

        user_item_matrix = user_item_matrix.astype(float)  # необходимый тип матрицы для implicit

        return user_item_matrix

    @staticmethod
    def _prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""

        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))

        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id

    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""

        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())

        return own_recommender

    @staticmethod
    def fit(user_item_matrix, n_factors=50, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""

        model = AlternatingLeastSquares(factors=n_factors,
                                        regularization=regularization,
                                        iterations=iterations,
                                        num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix).T.tocsr())

        return model

    def _update_dict(self, user_id):
        """Если появился новыю user / item, то нужно обновить словари"""

        if user_id not in self.userid_to_id.keys():

            max_id = max(list(self.userid_to_id.values()))
            max_id += 1

            self.userid_to_id.update({user_id: max_id})
            self.id_to_userid.update({max_id: user_id})

    def _get_similar_item(self, item_id):
        """Находит товар, похожий на item_id"""
        recs = self.model.similar_items(self.itemid_to_id[item_id], N=2)  # Товар похож на себя -> рекомендуем 2 товара
        top_rec = recs[1][0]  # И берем второй (не товар из аргумента метода)
        return self.id_to_itemid[top_rec]

    def _extend_with_top_popular(self, recommendations, N=5):
        """Если кол-во рекоммендаций < N, то дополняем их топ-популярными"""

        if len(recommendations) < N:
            recommendations.extend(self.overall_top_purchases[:N])
            recommendations = recommendations[:N]

        return recommendations

    def _get_recommendations(self, user, model, N=5):
        """Рекомендации через стардартные библиотеки implicit"""

        self._update_dict(user_id=user)
        res = [self.id_to_itemid[rec[0]] for rec in model.recommend(userid=self.userid_to_id[user],
                                        user_items=csr_matrix(self.user_item_matrix).tocsr(),
                                        N=N,
                                        filter_already_liked_items=False,
                                        filter_items=[self.itemid_to_id[999999]],
                                        recalculate_user=True)]

        res = self._extend_with_top_popular(res, N=N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

    def get_als_recommendations(self, user, N=5):
        """Рекомендации через стардартные библиотеки implicit"""

        self._update_dict(user_id=user)
        return self._get_recommendations(user, model=self.model, N=N)

    def get_own_recommendations(self, user, N=5):
        """Рекомендуем товары среди тех, которые юзер уже купил"""

        self._update_dict(user_id=user)
        return self._get_recommendations(user, model=self.own_recommender, N=N)

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        top_users_purchases = self.top_purchases[self.top_purchases['user_id'] == user].head(N)

        res = top_users_purchases['item_id'].apply(lambda x: self._get_similar_item(x)).tolist()
        res = self._extend_with_top_popular(res, N=N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""

        res = []

        # Находим топ-N похожих пользователей
        similar_users = self.model.similar_users(self.userid_to_id[user], N=N+1)
        similar_users = [rec[0] for rec in similar_users]
        similar_users = similar_users[1:]   # удалим юзера из запроса

        for user in similar_users:
            res.extend(self.get_own_recommendations(user, N=1))

        res = self._extend_with_top_popular(res, N=N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

# Filters

In [5]:
def prefilter_items(data, take_n_popular=5000, item_features=None):
    # Уберем самые популярные товары (их и так купят)
    popularity = data.groupby('item_id')['user_id'].nunique().reset_index() / data['user_id'].nunique()
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)

    top_popular = popularity[popularity['share_unique_users'] > 0.2].item_id.tolist()
    data = data[~data['item_id'].isin(top_popular)]

    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.02].item_id.tolist()
    data = data[~data['item_id'].isin(top_notpopular)]

    # Уберем не интересные для рекоммендаций категории (department)
    if item_features is not None:
        department_size = pd.DataFrame(item_features.\
                                        groupby('department')['item_id'].nunique().\
                                        sort_values(ascending=False)).reset_index()

        department_size.columns = ['department', 'n_items']
        rare_departments = department_size[department_size['n_items'] < 150].department.tolist()
        items_in_rare_departments = item_features[item_features['department'].isin(rare_departments)].item_id.unique().tolist()

        data = data[~data['item_id'].isin(items_in_rare_departments)]


    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб.
    data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))
    data = data[data['price'] > 2]

    # Уберем слишком дорогие товарыs
    data = data[data['price'] < 50]

    # Возьмем топ по популярности
    popularity = data.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

    top = popularity.sort_values('n_sold', ascending=False).head(take_n_popular).item_id.tolist()

    # Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
    data.loc[~data['item_id'].isin(top), 'item_id'] = 999999

    # ...

    return data


def postfilter(recommendations, item_info, N=5):
    """Пост-фильтрация товаров

    Input
    -----
    recommendations: list
        Ранжированный список item_id для рекомендаций
    item_info: pd.DataFrame
        Датафрейм с информацией о товарах
    """

    # Уникальность
#     recommendations = list(set(recommendations)) - неверно! так теряется порядок
    unique_recommendations = []
    [unique_recommendations.append(item) for item in recommendations if item not in unique_recommendations]

    # Разные категории
    categories_used = []
    final_recommendations = []

    CATEGORY_NAME = 'sub_commodity_desc'
    for item in unique_recommendations:
        category = item_features.loc[item_features['item_id'] == item, CATEGORY_NAME].values[0]

        if category not in categories_used:
            final_recommendations.append(item)

        unique_recommendations.remove(item)
        categories_used.append(category)

    n_rec = len(final_recommendations)
    if n_rec < N:
        final_recommendations.extend(unique_recommendations[:N - n_rec])
    else:
        final_recommendations = final_recommendations[:N]

    assert len(final_recommendations) == N, 'Количество рекомендаций != {}'.format(N)
    return final_recommendations

# Metrics

In [6]:
def precision_at_k(recommended_list, bought_list, k=5):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]

    flags = np.isin(bought_list, recommended_list)

    precision = flags.sum() / len(recommended_list)


    return precision

In [7]:
def ap_k(recommended_list, bought_list, k=5):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    flags = np.isin(recommended_list, bought_list)

    if sum(flags) == 0:
        return 0

    sum_ = 0
    for i in range(1, k+1):

        if flags[i - 1] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)

            sum_ += p_k

    result = sum_ / min(len(recommended_list), k)

    return result

In [8]:
def map_k(recommended_list_list, bought_list_list, k=5):

    ap = list()
    for i in range(len(recommended_list_list)):
        ap.append(ap_k(recommended_list_list[i], bought_list_list[i], k=k))

    map_metric = np.mean(ap)

    return map_metric

In [9]:
def ma_k_var2(predicted, actual, k=5):
    return np.mean([ap_k(a,p,k) for a,p in zip(actual, predicted)])

# Open data and their transform

In [10]:
data = pd.read_csv('./retail_train.csv')
item_features = pd.read_csv('./product.csv')
user_features = pd.read_csv('./hh_demographic.csv')
test_df = pd.read_csv('./retail_test1.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель --
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [11]:
users, items, interactions = data.user_id.nunique(), data.item_id.nunique(), data.shape[0]

print('# users: ', users) #пользователи
print('# items: ', items) #товары
print('# interactions: ', interactions) #взаимодействия

# users:  2499
# items:  89051
# interactions:  2396804


### Train-test split

In [ ]:
# Используем train-test split по времени, а не случайно. Возьмем последние 3 недели в качестве теста
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [ ]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

# 1. Бейзлайны

Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)

In [ ]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [ ]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


# 1.1 Popularity-based recommendation

In [ ]:
popularity = data.groupby('item_id')['user_id'].nunique().reset_index() / data['user_id'].nunique()
popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)

top_popular = popularity[popularity['share_unique_users'] > 0.3].item_id.tolist()


In [ ]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""

    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)

    recs = popular.head(n).item_id

    return recs.tolist()

In [ ]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

CPU times: user 113 ms, sys: 15.8 ms, total: 129 ms
Wall time: 130 ms


,user_id,actual,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534178, 6533889, 1029743, 6534166, 1082185]"


# 1.2. Main Recommender

In [ ]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


In [ ]:
recommender = MainRecommender(data_train_lvl_1)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [ ]:
recommender.get_als_recommendations(2375, N=5)

[871756, 899624, 854852, 1106523, 1044078]

In [ ]:
recommender.get_own_recommendations(2375, N=5)

[948640, 918046, 847962, 907099, 873980]

In [ ]:
recommender.get_similar_items_recommendation(2375, N=5)

[1046545, 1044078, 1042907, 824758, 6534480]

In [ ]:
recommender.get_similar_users_recommendation(2375, N=5)

[902640, 8090533, 1089568, 1072392, 921406]

In [ ]:
# В тестовом дата сетеесть "холодные" покупатели на которах модель не обучалась
# Найдем данные по таким покупателям, для них по умолчанию будем рекомендовать самые популярные товары
list_out = result.loc[~result.user_id.isin(data_train_lvl_1.user_id), 'user_id'].tolist()
list_out

[1984, 2259]

In [ ]:
def rule(x, y, model):
    if model == 'als':
        if x in y:
            return list([840361, 1029743, 995242, 981760, 1082185])
        else:
            return recommender.get_als_recommendations(x, N=5)
    elif model == 'own':
        if x in y:
            return list([840361, 1029743, 995242, 981760, 1082185])
        else:
            return recommender.get_own_recommendations(x, N=5)
    elif model == 'similar_items':
        if x in y:
            return list([840361, 1029743, 995242, 981760, 1082185])
        else:
            return recommender.get_similar_items_recommendation(x, N=5)
    elif model == 'similar_users':
        if x in y:
            return list([840361, 1029743, 995242, 981760, 1082185])
        else:
            return recommender.get_similar_users_recommendation(x, N=5)

In [ ]:
result['als'] = result['user_id'].apply(lambda x: rule(x, list_out, model='als'))

In [ ]:
result['own'] = result['user_id'].apply(lambda x: rule(x, list_out, model='own'))

In [ ]:
result['similar_items'] = result['user_id'].apply(lambda x: rule(x, list_out, model='similar_items'))

In [ ]:
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean() * 100

11.106758080313362

In [ ]:
result.apply(lambda row: precision_at_k(row['own'], row['actual']), axis=1).mean() * 100

14.427032321253568

In [ ]:
result.apply(lambda row: precision_at_k(row['similar_items'], row['actual']), axis=1).mean() * 100

5.954946131243923

In [ ]:
result.head(2)

,user_id,actual,popular_recommendation,als,own,similar_items
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6534178, 6533889, 1029743, 6534166, 1082185]","[891118, 883616, 856942, 9655212, 1025641]","[856942, 9297615, 5577022, 877391, 9655212]","[1022097, 1007512, 9297615, 5577022, 9803207]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534178, 6533889, 1029743, 6534166, 1082185]","[1075979, 998206, 1044078, 841762, 1056005]","[1092937, 1008714, 12132312, 1075979, 998206]","[1044078, 1075979, 883932, 839419, 865528]"


In [ ]:
# Для примера генерации в тестовый дата сет
# test_df['own'] = test_df['user_id'].apply(lambda x: rule(x, list_out, model='own'))

# Cross validation

In [ ]:
# user_item_matrix = recommender.user_item_matrix
# list_out = data_test.loc[~data_test.user_id.isin(data_train_lvl_1.user_id), 'user_id'].tolist()
# for factors in [50]:
#     recommender.fit(user_item_matrix, n_factors=factors, regularization=0.001, iterations=15, num_threads=4)
#     result['als'] = result['user_id'].apply(lambda x: rule(x, list_out, model='als'))

In [ ]:
# result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean() * 100

# Обучаем модель 2-ого уровня на выбранных кандидатах

In [ ]:
# -- давние покупки -- | -- 6 недель -- | -- 3 недель --

users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

# Пока только warm start
train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))
users_lvl_2.head(2)

,user_id,candidates
0,2070,"[1105426, 1097350, 879194, 948640, 928263, 944..."
1,2021,"[950935, 1119454, 835578, 863762, 1019142, 102..."


In [ ]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['flag'] = 1

users_lvl_2.head(4)

,user_id,item_id,flag
0,2070,1105426,1
0,2070,1097350,1
0,2070,879194,1
0,2070,948640,1


In [ ]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('flag', axis=1, inplace=True)
targets_lvl_2.head(2)

,user_id,item_id,target
0,2070,1105426,0.0
1,2070,1097350,0.0


In [ ]:
df = data_train.groupby('user_id')['item_id'].apply(list).reset_index()
print (df)

      user_id                                            item_id
0           1  [825123, 831447, 840361, 845307, 852014, 85498...
1           2  [854852, 930118, 1077555, 1098066, 5567388, 55...
2           3  [866211, 878996, 882830, 904360, 921345, 93194...
3           4  [836163, 857849, 877523, 878909, 883932, 89142...
4           5  [938983, 5980822, 1012352, 825538, 1002499, 69...
...       ...                                                ...
2494     2496  [840361, 852159, 871756, 886703, 899624, 91612...
2495     2497  [838220, 1037840, 1052294, 5569230, 8090537, 1...
2496     2498  [824555, 835576, 901776, 904023, 911215, 91749...
2497     2499  [838186, 853197, 864143, 883665, 932949, 93383...
2498     2500  [856455, 903476, 834102, 837751, 880797, 89301...

[2499 rows x 2 columns]


In [ ]:
# Фильтрация полученных рекомендаций
# result = [postfilter(recommendations, item_info=item_features, N=5) for recommendations in df['item_id']]

# 1.0. Filter Items

In [ ]:
# n_items_before = data_train['item_id'].nunique()

# data_train = prefilter_items(data_train, item_features=item_features)

# n_items_after = data_train['item_id'].nunique()
# print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

# 1.2. Возьмем 1 наиболее близкого соседа

In [ ]:
# popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
# popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

# popularity.head()

In [ ]:
# top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [ ]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
# data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id', columns='item_id',
                                  values='quantity',
                                  aggfunc='count',
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,25671,26081,26093,26190,26355,26426,26540,26601,26636,26691,...,17328742,17329473,17329749,17330255,17330511,17381856,17382205,17383227,17827644,17829232
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [ ]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во ближайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),
          show_progress=True)

  0%|          | 0/86865 [00:00<?, ?it/s]

CPU times: user 20.9 s, sys: 149 ms, total: 21 s
Wall time: 10.5 s


In [ ]:
%%time

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in
                    model.recommend(userid=userid_to_id[x],
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5,
                                    filter_already_liked_items=False,
                                    filter_items=None,
                                    recalculate_user=False)])

CPU times: user 98.8 ms, sys: 916 µs, total: 99.7 ms
Wall time: 99.4 ms


In [ ]:
result.head(5)

,user_id,actual,popular_recommendation,als,own,similar_items,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6534178, 6533889, 1029743, 6534166, 1082185]","[891118, 883616, 856942, 9655212, 1025641]","[856942, 9297615, 5577022, 877391, 9655212]","[1022097, 1007512, 9297615, 5577022, 9803207]","[1082185, 995242, 1029743, 840361, 904360]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534178, 6533889, 1029743, 6534166, 1082185]","[1075979, 998206, 1044078, 841762, 1056005]","[1092937, 1008714, 12132312, 1075979, 998206]","[1044078, 1075979, 883932, 839419, 865528]","[1082185, 1098066, 6534178, 826249, 1127831]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6534178, 6533889, 1029743, 6534166, 1082185]","[965267, 1127179, 896613, 951834, 898958]","[13003092, 972416, 995598, 923600, 1138596]","[948650, 5569845, 823990, 878996, 999999]","[1082185, 981760, 995242, 1029743, 1127831]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[6534178, 6533889, 1029743, 6534166, 1082185]","[5981267, 10285022, 854042, 839419, 871513]","[998519, 894360, 7147142, 9338009, 896666]","[839419, 1044078, 982469, 6602697, 896027]","[1082185, 995242, 1029743, 1127831, 826249]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[6534178, 6533889, 1029743, 6534166, 1082185]","[998236, 1034838, 7144132, 1050645, 12352054]","[12808385, 981660, 939860, 7410201, 6463874]","[5569845, 954966, 1044078, 906905, 872137]","[1082185, 981760, 995242, 1029743, 1098066]"


# 1.3. Alternating Least Squares (ALS)

In [ ]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in
                    model.recommend(userid=userid_to_id[user],
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N,
                                    filter_already_liked_items=False,
                                    filter_items=None,
                                    recalculate_user=True)]
    return res

In [ ]:
%%time

model = AlternatingLeastSquares(factors=50,
                                regularization=0.001,
                                iterations=15,
                                calculate_training_loss=True,
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_var'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 4min 43s, sys: 3min 39s, total: 8min 22s
Wall time: 2min 12s


# 1.3. TF-IDF взвешивание

In [ ]:
user_item_matrix = tfidf_weight(user_item_matrix.T).T  # Применяется к item-user матрице !

In [ ]:
%%time

model = AlternatingLeastSquares(factors=50,
                                regularization=0.001,
                                iterations=15,
                                calculate_training_loss=True,
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_tfidf'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 4min 43s, sys: 3min 41s, total: 8min 25s
Wall time: 2min 8s


# 1.4. BM25 взвешивание

In [ ]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
# data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id', columns='item_id',
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count',
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,25671,26081,26093,26190,26355,26426,26540,26601,26636,26691,...,17328742,17329473,17329749,17330255,17330511,17381856,17382205,17383227,17827644,17829232
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице !

In [ ]:
%%time

model = AlternatingLeastSquares(factors=50,
                                regularization=0.001,
                                iterations=15,
                                calculate_training_loss=True,
                                num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 4min 43s, sys: 3min 41s, total: 8min 25s
Wall time: 2min 8s


# Filter Items and model LFM

In [ ]:
# n_items_before = data_train['item_id'].nunique()

# data_train = prefilter_items(data_train, item_features=item_features)

# n_items_after = data_train['item_id'].nunique()
# print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

In [ ]:
# user_item_matrix = pd.pivot_table(data_train,
#                                   index='user_id', columns='item_id',
#                                   values='quantity', # Можно пробоват ьдругие варианты
#                                   aggfunc='count',
#                                   fill_value=0
#                                  )

# user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# # переведем в формат saprse matrix
# sparse_user_item = csr_matrix(user_item_matrix).tocsr()

# user_item_matrix.head(2)

In [ ]:
# data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]
# test_user_item_matrix = pd.pivot_table(data_test,
#                                   index='user_id', columns='item_id',
#                                   values='quantity', # Можно пробоват ьдругие варианты
#                                   aggfunc='count',
#                                   fill_value=0
#                                  )

# test_user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [ ]:
# userids = user_item_matrix.index.values
# itemids = user_item_matrix.columns.values

# matrix_userids = np.arange(len(userids))
# matrix_itemids = np.arange(len(itemids))

# id_to_itemid = dict(zip(matrix_itemids, itemids))
# id_to_userid = dict(zip(matrix_userids, userids))

# itemid_to_id = dict(zip(itemids, matrix_itemids))
# userid_to_id = dict(zip(userids, matrix_userids))

In [ ]:
# user_feat = pd.DataFrame(user_item_matrix.index)
# user_feat = user_feat.merge(user_features, on='user_id', how='left')
# user_feat.set_index('user_id', inplace=True)

# item_feat = pd.DataFrame(user_item_matrix.columns)
# item_feat = item_feat.merge(item_features, on='item_id', how='left')
# item_feat.set_index('item_id', inplace=True)

# user_feat.head(2)

In [ ]:
# user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
# item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [ ]:
# user_feat_lightfm.head(2)

In [ ]:
# model = LightFM(no_components=30,
#                 loss='bpr', # 'warp'
#                 learning_rate=0.05,
#                 item_alpha=0.1, user_alpha=0.1,
#                 random_state=42)

# model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
#           sample_weight=coo_matrix(user_item_matrix),
#           user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
#           item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
#           epochs=15,
#           num_threads=4)

In [ ]:
# user_emb = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values).tocsr())

In [ ]:
# train_precision = precision_at_k(model, sparse_user_item,
#                                  user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
#                                  item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
#                                  k=5).mean()

# train_precision

# Измерим качество по precision@5

In [ ]:
result.head(2)

,user_id,actual,popular_recommendation,als,own,similar_items,own_purchases,als_var,als_tfidf,als_bm25
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6534178, 6533889, 1029743, 6534166, 1082185]","[891118, 883616, 856942, 9655212, 1025641]","[856942, 9297615, 5577022, 877391, 9655212]","[1022097, 1007512, 9297615, 5577022, 9803207]","[1082185, 995242, 1029743, 840361, 904360]","[1082185, 1127831, 5978656, 1033142, 1024306]","[1082185, 1100972, 995242, 981760, 5978656]","[1082185, 1127831, 856942, 5978656, 1075074]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534178, 6533889, 1029743, 6534166, 1082185]","[1075979, 998206, 1044078, 841762, 1056005]","[1092937, 1008714, 12132312, 1075979, 998206]","[1044078, 1075979, 883932, 839419, 865528]","[1082185, 1098066, 6534178, 826249, 1127831]","[951590, 5569230, 826249, 910032, 1098066]","[883404, 1082185, 951590, 1029743, 1106523]","[1044078, 826249, 1098066, 981760, 883404]"


In [ ]:
result.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual']), axis=1).mean() * 100

15.523996082272081

In [ ]:
result.apply(lambda row: precision_at_k(row['own_purchases'], row['actual']), axis=1).mean() * 100

21.949069539666716

In [ ]:
result.apply(lambda row: precision_at_k(row['als_var'], row['actual']), axis=1).mean() * 100

20.7443682664052

In [ ]:
result.apply(lambda row: precision_at_k(row['als_tfidf'], row['actual']), axis=1).mean() * 100

21.508325171400315

In [ ]:
result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean() * 100

19.862879529872433

# Измерим качество по MAP@k (Mean Average Precision@k)  
- Показывает средневзвешенную точность рекомендаций

In [ ]:
result.apply(lambda row: ap_k(row['popular_recommendation'], row['actual']), axis=1).mean() * 100

8.280770486451184

In [ ]:
result.apply(lambda row: ap_k(row['own_purchases'], row['actual']), axis=1).mean() * 100

15.933398628795272

In [ ]:
result.apply(lambda row: ap_k(row['als'], row['actual']), axis=1).mean() * 100

6.96833170094677

In [ ]:
result.apply(lambda row: ap_k(row['als_tfidf'], row['actual']), axis=1).mean() * 100

15.39895527260852

In [ ]:
result.apply(lambda row: ap_k(row['als_bm25'], row['actual']), axis=1).mean() * 100

14.645445641527873

# 8. Прогнозирование на тестовом датасете

Для удобства перевода данных в подходящий для загрузки формат:

In [ ]:
def transform_data_for_eval(dataset, rec_col, user_col='user_id'):
    '''
    Func for transforming recommendations into kaggle evaluation format

    Parameters:
    dataset (pd.DataFrame): Dataset with 2 required columns:
        rec_col - column with recommendations should be iterable
        user_col - columns with user id

    rec_col (str): name of column in dataset with recommendations

    user_col (str): name of column in dataset with user id

    Returns:
    pd.DataFrame: DataFrame in suitable format

   '''
    eval_dataset = dataset[[user_col, rec_col]].copy()
    eval_dataset[rec_col] = eval_dataset[rec_col].apply(lambda x: ' '.join([str(i) for i in x]))
    eval_dataset.rename(columns={
        user_col: 'UserId',
        rec_col: 'Predicted'
    }, inplace=True)
    return eval_dataset

In [ ]:
submit = pd.read_csv('/kaggle/input/gb-recsys-project/sample_submission.csv')
submit.head()

,UserId,Predicted
0,1,1082185 981760 995242 1029743 840361
1,2,1082185 981760 995242 1029743 840361
2,3,1082185 981760 995242 1029743 840361
3,6,1082185 981760 995242 1029743 840361
4,7,1082185 981760 995242 1029743 840361


# Эффективней других оказалась идея найти 1 ближашего соседа и фильтровать по топ 5000, ее и применим.

In [ ]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in
                    model.recommend(userid=userid_to_id[user],
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N,
                                    filter_already_liked_items=False,
                                    filter_items=None,
                                    recalculate_user=True)]
    return res

In [ ]:
user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id', columns='item_id',
                                  values='quantity',
                                  aggfunc='count',
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [ ]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [ ]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во ближайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),
          show_progress=True)

  0%|          | 0/86865 [00:00<?, ?it/s]

CPU times: user 20.5 s, sys: 157 ms, total: 20.6 s
Wall time: 10.3 s


In [ ]:
model

In [ ]:
list_out = test_df.loc[~test_df.user_id.isin(data_train.user_id), 'user_id'].tolist()
list_out

[2325]

In [ ]:
def rule(x, y):
    if x in y:
         return list([840361, 1029743, 995242, 981760, 1082185])
    else:
         return get_recommendations(x, model=model, N=5)

In [ ]:
test_df['own_purchases'] = test_df['user_id'].apply(lambda x: rule(x, list_out))

In [ ]:
test_df.head(2)

,user_id,own_purchases
0,1,"[1082185, 995242, 1029743, 840361, 904360]"
1,2,"[1082185, 1127831, 1098066, 840361, 826249]"


In [ ]:
rec_df = transform_data_for_eval(test_df, rec_col='own_purchases', user_col='user_id')

In [ ]:
rec_df.head(2)

,UserId,Predicted
0,1,1082185 995242 1029743 840361 904360
1,2,1082185 1127831 1098066 840361 826249


In [ ]:
rec_df.to_csv(' recommendations.csv', index=False)